# 0. Example format

In [7]:
### This dummy dataframe depicts the format of the input data
### The labels are are provided in a one-hot encoded fashion and as such mutually exclusive.
import pandas as pd

d = {'Diagnosis_1': [0, 0, 0, 1, 1, 1, 0, 0, 0, 0], 
     'Diagnosis_2': [0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
     'Diagnosis_3': [1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
     'Report': ['free text']*10}
df = pd.DataFrame(data=d)

df

,Diagnosis_1,Diagnosis_2,Diagnosis_3,Report
0,0,0,1,free text
1,0,1,0,free text
2,0,0,1,free text
3,1,0,0,free text
4,1,0,0,free text
5,1,0,0,free text
6,0,0,1,free text
7,0,1,0,free text
8,0,0,1,free text
9,0,1,0,free text


# 1. Importing Data

In [3]:
import pandas as pd
import os

# Set directories
# source_dir = ### ENTER SOURCE DIRECTORY ###

# Import Excel file
df = pd.read_excel(os.path.join(source_dir,'df_one_hot_encoded.xlsx'))

df[['Glioma', 'Meningioma', 'Metastasis', 'Report']].head(10)

,Glioma,Meningioma,Metastasis,Report
0,1,0,0,|Accession Number: BL15A20488 ...
1,1,0,0,|Accession Number: BL15A34100 ...
2,1,0,0,|Accession Number: BL15D23219 ...
3,1,0,0,|Accession Number: BL15D38566 ...
4,1,0,0,|Accession Number: BL15D43181 ...
5,1,0,0,|Accession Number: BL15E22467 ...
6,1,0,0,|Accession Number: BL15E47811 ...
7,0,0,1,|Accession Number: BL15F23188 ...
8,1,0,0,|Accession Number: BL15F24113 ...
9,1,0,0,|Accession Number: BL15F25117 ...


# 2. Preprocessing

### 2A. Shuffle data

In [ ]:
df = df.sample(frac=1)
df.head(2)

### 2B. Remove uneccesary patterns

In [ ]:
import re

def preprocessor(text):
    
    #remove all text before 'CLINICAL DATA'
    text = re.sub(r'.*CLINICAL DATA', 'CLINICAL DATA', text)
    
    #remove disclaimer and signature'
    text = text.split('These tests were developed and their performance characteristics determined bythe Molecular Diagnostics Laboratory')[0]
    
    # Remove newlines
    text = text.replace(r'\n', ' ')
    
    # Remove date
    date_pattern = r'[0-9]{1,2}[-/][0-9]{1,2}[-/][0-9]{2,}'
    text = re.sub(date_pattern, ' ', text.lower())
    
    # Remove whitespace
    text = ' '.join(text.split())
    
    # Remove punctution, keep decimal points
    text = re.sub(r'[\W]+(?!\d)', ' ', text)

    return text

df['cleaned_report'] = df['Report'].apply(preprocessor)
df['cleaned_report'][1]

### 2C. Remove stop words

In [ ]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words_english = stopwords.words('english')

stop_words_modified = [w for w in stop_words_english if w not in ['no', 'not']]

def remove_words(clean_report): 
    clean_report_words = clean_report.split()
    stripped_report_words  = [word for word in clean_report_words if word.lower() not in stop_words_modified]
    stripped_report_text = ' '.join(stripped_report_words)
    return stripped_report_text

df['stripped_report'] = df['cleaned_report'].apply(remove_words)
df.head(2)

### 2D. Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def porter_stemming(text):
    stemmed_report_words = [porter.stem(word) for word in text.split()]
    stemmed_report_text = ' '.join(stemmed_report_words)
    return stemmed_report_text

df['stemmed_report'] = df['stripped_report'].apply(porter_stemming)
df.head(2)

### 2E. Tokenizing

In [ ]:
def split_words(report): 
    report = report.split()
    return report

df['tokenized_report'] = df['stemmed_report'].apply(split_words)

df.head(2)

### 2F.  Train validation test split

In [ ]:
import pandas as pd
import os
import numpy as np
import random

n_patients = len(df)

df = df.sample(frac = 1)
list_train_test = ['train']*3000+['validation']*2000+['test']*2000
df['cohort'] =  random.sample(list_train_test, len(list_train_test))

df

### 2F. Save Reports

In [ ]:
target_dir = ### ENTER TARGET DIRECTORY ###
df.to_excel(os.path.join(target_dir, 'preprocessed_reports.xlsx')) 

### 2H. Load Preprocessed Reports 

In [ ]:
import pandas as pd
import os

source_dir = ### ENTER SOURCE DIRECTORY ###
df = pd.read_excel(os.path.join(source_dir, 'preprocessed_reports.xlsx')) 
print('Dimensions df are', df.shape)
df.head(5)

# 3 Statistical / Classic Machine Learning

### 3A. Create an parameter grid

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid_dict = {'diagnosis': ['Glioma', 'Meningioma', 'Metastasis'],
                    'l1': [1, 2, 4, 8, 16],
                   'max_features':[100, 250, 500, 1000, 1500, 2000],
                   'ngram_range': [1, 2, 3],
                   'sample_size': [25, 50, 75, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1200, 1500, 1800, 2100, 2500, 3000] 
                  }

param_grid_list = list(ParameterGrid(param_grid_dict))
param_grid_list

parameter_grid = []
for dict in param_grid_list:
    hyperparameters = list(dict.values())
    parameter_grid.append(hyperparameters)

print('Length parameter grid is', len(parameter_grid), 'hyperparameter settings')
parameter_grid

### 3D. Hyperparameter tuning regression-based models

In [ ]:
### Running this code block on a multiple cores or a cloud service might be desirable

from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Lasso 
from sklearn.metrics import roc_auc_score
from sklearn import linear_model
import scipy.sparse as sp
import pandas as pd
import numpy as np
import math
import os

def square(list):
    return [i ** 2 for i in list]

diagnosis_column = []
l1_column = []
max_features_column = []
ngram_range_column = []
sample_size_column = []
auc_mean_column = []
auc_std_column = []
number_of_bootstraps = []

start_gridsearch = 1

for diagnosis, l1, max_features, ngram_range, sample_size in parameter_grid:
    
    print('##### Grid Search', start_gridsearch,'/', len(parameter_grid))
    
    word_vectorizer = TfidfVectorizer(ngram_range=(1,ngram_range),max_features=max_features)
    word_vectorizer.fit(df['tokenized_report'][df['cohort'].isin(['train', 'validation'])])
  
    bootstrap_aucs = []
    
    start_bootstrap = 1
    
    for n in range(int(3000//sample_size)):
        
        df_frac = df[(df['cohort'] == 'train')].sample(n = sample_size, replace = True)
    
        X_train = word_vectorizer.transform(df_frac['tokenized_report']).toarray()
        y_train = np.asarray(df_frac[diagnosis])
        
        X_val = word_vectorizer.transform(df['tokenized_report'][df['cohort'].isin(['validation'])]).toarray()
        y_val = np.asarray(df[diagnosis][df['cohort'].isin(['validation'])])
        
        try:
            model_lasso = LogisticRegression(penalty = 'l1', C=l1, max_iter=200)
            model_lasso.fit(X_train, y_train)

            bootstrap_auc = roc_auc_score(y_val, model_lasso.predict_proba(X_val)[:,1])
            bootstrap_aucs.append(bootstrap_auc)
        
        except ValueError:
               pass
                                            
        start_bootstrap = start_bootstrap + 1 
                                          
    n_bootstraps = start_bootstrap - 1
    
    print('Number of bootstraps is', n_bootstraps)                                     
                                          
    auc_mean = np.mean(bootstrap_aucs)
    auc_std = np.std(bootstrap_aucs)

    print('The mean bootstrapped AUC for grid search', start_gridsearch, 'bootstrap ', start_bootstrap,
          round(auc_mean,2),'±', round(auc_std,3))
    
    diagnosis_column.append(diagnosis)
    l1_column.append(l1)
    max_features_column.append(max_features)
    ngram_range_column.append(ngram_range)
    sample_size_column.append(sample_size)
    auc_mean_column.append(auc_mean)
    auc_std_column.append(auc_std)
    number_of_bootstraps.append(n_bootstraps)
    
    start_gridsearch = start_gridsearch + 1
    
df_performance = pd.DataFrame(list(zip(diagnosis_column, sample_size_column, l1_column, max_features_column, ngram_range_column,
                                    auc_mean_column, auc_std_column, number_of_bootstraps)))

df_performance.columns = ['diagnosis', 'sample_size', 'l1', 'max_features', 'ngram_range',  'auc_mean', 
                          'auc_std', 'number_of_bootstraps']

target_dir = ### ENTER TARGET DIRECTORY ###
df_performance.to_excel(os.path.join(target_dir, 'hyperparameter_tuning.xlsx'))

df_performance

### 3D. Select optimal hyperparameter settings

In [ ]:
import pandas as pd
import os
import numpy as np

source_dir =  ### ENTER SOURCE DIRECTORY ###
df_hyperparameter_results = pd.read_excel(os.path.join(source_dir, 'iteration_1.xlsx'))

df_hyperparameter_optimal_lasso = df_hyperparameter_results.loc[df_hyperparameter_results.reset_index().groupby(['diagnosis', 'sample_size'])['auc_mean'].idxmax()]
df_hyperparameter_optimal_lasso = df_hyperparameter_optimal_lasso[['diagnosis', 'l1', 'max_features', 'ngram_range', 'sample_size']]
optimal_paramater_grid_lasso = df_hyperparameter_optimal_lasso.values.tolist()
optimal_paramater_grid_lasso

df_hyperparameter_results_logistic = df_hyperparameter_results.loc[df_hyperparameter_results['l1'] == 1]
df_hyperparameter_optimal_logistic = df_hyperparameter_results_logistic.loc[df_hyperparameter_results_logistic.groupby(['diagnosis', 'sample_size'])['auc_mean'].idxmax()]
df_hyperparameter_optimal_logistic = df_hyperparameter_optimal_logistic[['diagnosis', 'l1', 'max_features', 'ngram_range', 'sample_size']]
optimal_paramater_grid_logistic = df_hyperparameter_optimal_logistic.values.tolist()
optimal_paramater_grid_lasso

### 3E. Train final models and compute predicted probabilities and classses

In [ ]:
### Run this code block twice. Once for Lasso, once for logistic regression.

from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Lasso 
from sklearn.metrics import roc_auc_score
from keras.utils import to_categorical
from sklearn import linear_model
import scipy.sparse as sp
import pandas as pd
import numpy as np
import os

def square(list):
    return [i ** 2 for i in list]

diagnosis_column = []
l1_column = []
max_features_column = []
ngram_range_column = []
sample_size_column = []
auc_mean = []
auc_std = []
number_of_bootstraps = []

gridsearch = 1

for diagnosis, l1, max_features, ngram_range, sample_size in optimal_paramater_grid_lasso:
    
    print('##### Grid Search', gridsearch,'/', len(optimal_paramater_grid_lasso))
    
    word_vectorizer = TfidfVectorizer(ngram_range=(1,ngram_range),max_features=max_features)
    word_vectorizer.fit(df['tokenized_report'][df['cohort'].isin(['train', 'test'])])

    bootstrap_aucs = [] 
    
    start_bootstrap = 1
    
    for n in range(100):
        
        df_frac = df[(df['cohort'] == 'train')].sample(n = sample_size, replace = True)
    
        X_train = word_vectorizer.transform(df_frac['tokenized_report']).toarray()
        y_train = np.asarray(df_frac[diagnosis])
        
        X_test = word_vectorizer.transform(df['tokenized_report'][df['cohort'].isin(['test'])]).toarray()
        y_test = np.asarray(df[diagnosis][df['cohort'].isin(['test'])])
        
        try:
            model = LogisticRegression(penalty = 'l1', C=l1, max_iter=200)
            model.fit(X_train, y_train)

            predicted_probabilities = model.predict_proba(X_test)[:,1]
            auc = roc_auc_score(y_test, predicted_probabilities)
            print('AUC', auc)

        except ValueError:
              pass
        
        print('AUC:', str(round(auc,3)))
        bootstrap_aucs.append(auc)
        
        gridsearch += 1 
                                          
    n_bootstraps = gridsearch - 1
    print('### Number of bootstraps is', n_bootstraps)
    
    bootstrap_auc_mean = np.mean(bootstrap_aucs)
    bootstrap_auc_std = np.std(bootstrap_aucs)
    print('### Mean AUC for grid search', gridsearch, ':', round(bootstrap_auc_mean,2),'±', round(bootstrap_auc_std,3))

    print('')
    
    diagnosis_column.append(diagnosis)
    l1_column.append(l1)
    max_features_column.append(max_features)
    ngram_range_column.append(ngram_range)
    sample_size_column.append(sample_size)
    auc_mean.append(bootstrap_auc_mean)
    auc_std.append(bootstrap_auc_std)
    number_of_bootstraps.append(n_bootstraps)
    
    start_gridsearch = start_gridsearch + 1
    
df_performance = pd.DataFrame(list(zip(diagnosis_column, sample_size_column, l1_column, max_features_column, ngram_range_column,
                                    auc_mean, auc_std, number_of_bootstraps)))

df_performance.columns = ['diagnosis', 'sample_size', 'l1', 'max_features', 'ngram_range',  'auc_mean', 
                          'auc_std', 'number_of_bootstraps']

target_dir = ### ENTER TARGET DIRECTORY ###

df_performance.to_excel(os.path.join(target_dir, 'final_results_lasso.xlsx'))

df_performance
                            

### 3F. Get results lasso regression model

In [19]:
import pandas as pd
import os

path_final_results = ### ENTER SOURCE DIRECTORY ###
df_results_lasso = pd.read_excel(os.path.join(path_final_results, 'pooled_final_results_lasso.xlsx'))

df_results_lasso[['sample_size', 'pooled_auc','pooled_auc_std', 'auc_mean_glioma', 'auc_std_glioma', 'auc_mean_meningioma', 'auc_std_glioma', 'auc_mean_metastasis', 'auc_std_metastasis']]

,sample_size,pooled_auc,pooled_auc_std,auc_mean_glioma,auc_std_glioma,auc_mean_meningioma,auc_std_glioma,auc_mean_metastasis,auc_std_metastasis
0,25,0.712994,0.098449,0.645773,0.072372,0.800798,0.072372,0.692411,0.110357
1,50,0.835223,0.068722,0.773859,0.072240,0.891660,0.072240,0.840152,0.081601
2,75,0.899991,0.034488,0.897357,0.032403,0.913687,0.032403,0.888931,0.039001
3,100,0.928525,0.018110,0.934140,0.016937,0.927562,0.016937,0.923872,0.019718
4,150,0.948871,0.012130,0.954875,0.011957,0.942720,0.011957,0.949018,0.011717
5,200,0.958270,0.010108,0.966532,0.005436,0.948886,0.005436,0.959393,0.011541
6,250,0.964613,0.007218,0.970602,0.004577,0.957131,0.004577,0.966105,0.007668
7,300,0.967522,0.006599,0.969216,0.004773,0.962033,0.004773,0.971317,0.005340
8,400,0.973507,0.005944,0.977039,0.003113,0.968108,0.003113,0.975375,0.005917
9,500,0.975880,0.003988,0.977130,0.002884,0.971987,0.002884,0.978522,0.003370


### 3G. Get results logistic regression model

In [21]:
import pandas as pd
import os

path_final_results = ### ENTER SOURCE DIRECTORY ###
df_results_logistic = pd.read_excel(os.path.join(path_final_results, 'pooled_final_results_logistic.xlsx'))

df_results_logistic[['sample_size', 'pooled_auc','pooled_auc_std', 'auc_mean_glioma', 'auc_std_glioma', 'auc_mean_meningioma', 'auc_std_glioma', 'auc_mean_metastasis', 'auc_std_metastasis']]

,sample_size,pooled_auc,pooled_auc_std,auc_mean_glioma,auc_std_glioma,auc_mean_meningioma,auc_std_glioma,auc_mean_metastasis,auc_std_metastasis
0,25,0.509732,0.025537,0.515867,0.031513,0.505776,0.031513,0.507555,0.024826
1,50,0.570702,0.101360,0.531664,0.041959,0.627722,0.041959,0.552719,0.052011
2,75,0.681322,0.099870,0.643186,0.080922,0.792921,0.080922,0.607859,0.053303
3,100,0.784374,0.083353,0.701061,0.068212,0.886266,0.068212,0.765797,0.113163
4,150,0.869661,0.042154,0.832498,0.042225,0.907103,0.042225,0.869381,0.054945
5,200,0.914377,0.022744,0.930762,0.024465,0.922036,0.024465,0.890333,0.024185
6,250,0.932040,0.010949,0.951406,0.009800,0.934822,0.009800,0.909892,0.011508
7,300,0.938912,0.012552,0.955914,0.008471,0.936237,0.008471,0.924583,0.016552
8,400,0.952147,0.009749,0.964059,0.004845,0.946910,0.004845,0.945474,0.014719
9,500,0.961622,0.006677,0.971226,0.003465,0.952100,0.003465,0.961539,0.008441


# 4. Deep Learning

### 4A. Load preprocessed reports

In [ ]:
import pandas as pd
import os

source_dir = ### ENTER SOURCE DIRECTORY ###
df = pd.read_excel(os.path.join(source_dir, 'preprocessed_reports.xlsx')) 
print('Dimensions df are', df.shape)
df.head(10)

### 4B. Create an parameter grid deep learning model

In [ ]:
# In total, we tested 58 settings across 15 hyperparameters resulting in a hyperparameter space 
# that encompasses 1,300,561,920 hypothetical model architectures.
# Because it is computationally impossible to evaluate all these model architectures, 
# we applied an hierarchical approach for hyperparameter optimization by subsequently optimizing the:
# 1) preprocessing settings (embedding_dim, max_len, max_words)
# 2) network architecture (double_layer, filter_size, n_filters_conv, n_filters_dense, n_layers_conv, n_layers_dense)
# 3) regularizers (dropout, l1, l2, max_pooling)
# 4) learning speed (optimizer_type, learning_rate)

from sklearn.model_selection import ParameterGrid

param_grid_dict = { 
    'double_layer': [0,1], 
    'dropout': [0, 0.1, 0.2, 0.5], 
    'embedding_dim': [8, 16, 32, 64, 128, 256, 512],
    'filter_size': [1,3,5,7,9], 
    'l1': [0.00000, 0.00001, 0.0001, 0.001],
    'l2': [0.00000, 0.00001, 0.0001, 0.001], 
    'learning_rate': [0.002, 0.005, 0.01, 0.05], 
    'max_len': [250, 500, 1000, 2000],
    'max_pooling' : [0,1],
    'max_words': [250, 500, 750, 1000, 1500, 2000, 5000],
    'n_filters_conv:': [8, 16, 32, 64],
    'n_filters_dense':[8,16,32],
    'n_layers_conv': [0,1,2,3],
    'n_layers_dense':[0,1],
    'optimizer_type': ['adam', 'nadam'],
    'sample_size': [25, 50, 100, 150, 200, 250, 300, 400, 500, 750, 1000, 1500, 2000, 3000],
                  }

param_grid_list = list(ParameterGrid(param_grid_dict))

parameter_grid = []
for dict in param_grid_list:
    hyperparameters = list(dict.values())
    parameter_grid.append(hyperparameters)

print('Length parameter grid is', len(parameter_grid), 'hyperparameter settings')

parameter_grid

###  4C. Hyperparameter tuning deep learning

In [ ]:
### In this code block, two functions were used to automatically construct and evaluate different model architectures.
### Running this code block on a multiple cores or a cloud service might be desirable.
### Due to the hierarchical approach of the hyperparameter optimization,
### the best model architecture is not automatically extracted 
### but retrieved through and iterative proces.

from keras.layers import Embedding, Flatten, Dense, Input, LSTM, Conv1D, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split, KFold
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import roc_auc_score
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import preprocessing
from keras import regularizers
from keras import optimizers
from keras import layers
from numpy import array
import pandas as pd
import numpy as np
import math
import time

bootstrap_time_column = []
n_layers_dense_column = []
double_layer_column = []
dropout_column = []
embedding_dim_column = []
filter_size_column = []
l1_column = []
l2_column = []
learning_rate_column = []
max_len_column = []
max_pooling_column = []
max_words_column = []
n_filters_dense_column = []
n_filters_conv_column = []
n_layers_conv_column = []
n_params_column = []
optimizer_type_column = []
sample_size_column = []
auc_mean = []
auc_std = []
epochmeans = []
epochstds = []

gridsearch = 1


def get_conv1d_model(n_layers_conv, double_layer, n_filters_conv, filter_size, l1, l2, dropout):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    if n_layers_conv == 0:
        model.add(layers.MaxPooling1D(filter_size))
    elif n_layers_conv == 1:
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        model.add(layers.MaxPooling1D(filter_size))
    elif n_layers_conv == 2:
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        model.add(layers.MaxPooling1D(filter_size))
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))            
        model.add(layers.MaxPooling1D(filter_size))
    elif n_layers_conv == 3:
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        model.add(layers.MaxPooling1D(filter_size))
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))             
        model.add(layers.MaxPooling1D(filter_size))
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv*4, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv*4, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv*4, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        model.add(layers.MaxPooling1D(filter_size))
    model.add(layers.GlobalMaxPooling1D())
    model.add(Dropout(dropout))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['acc'])
    return model

def get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(dropout))
    if n_layers_conv == 1:
        model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
    if max_pooling == 1:
        model.add(layers.MaxPooling1D(filter_size))
    model.add(layers.GlobalMaxPooling1D())
    if n_layers_dense == 1:
        model.add(Dense(n_filters_dense, activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer=final_optimizer, loss='categorical_crossentropy', metrics=['acc'])
    return model

def square(list):
    return [i ** 2 for i in list]

df_train_val = df[df['cohort'].isin(['train', 'validation'])]

for double_layer, dropout, embedding_dim, filter_size, l1, l2, learning_rate, max_len, max_pooling, max_words, n_filters_conv, n_filters_dense, n_layers_conv, n_layers_dense, optimizer_type, sample_size in parameter_grid:
    
    print('##### Grid Search', gridsearch,'/', len(parameter_grid))
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(df_train_val['stemmed_report'])
    sequences = tokenizer.texts_to_sequences(df_train_val['stemmed_report'])
    
    df_train_val['vectorized_report'] = sequences 
    
    bootstrap_aucs = []
    bootstrap_auc_stds = []
    
    bootstrap_epoch = []
    bootstrap_epoch_std = []
    
    bootstrap = 1

    if optimizer_type == 'adam':
        final_optimizer = optimizers.Adam(lr=learning_rate)
    elif optimizer_type == 'nadam':
        final_optimizer = optimizers.Nadam(lr=learning_rate)
    
    #model = get_conv1d_model(n_layers_conv, double_layer, n_filters_conv, filter_size, l1, l2, dropout)
    model = get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer)
    model.summary()

    start_time = time.time()
          
    for n in range(int(3000//sample_size)): 
        
        print('--- Bootstrap', bootstrap)

        df_frac = df_train_val[(df_train_val['cohort'] == 'train')].sample(n = 2*sample_size, replace = True)

        array_train = array(list(df_frac['vectorized_report'][(df_frac['cohort'] == 'train')]))
        array_val = array(list(df_train_val['vectorized_report'][(df_train_val['cohort'] == 'validation')]))
        X_train = preprocessing.sequence.pad_sequences(array_train, maxlen=max_len)
        X_val = preprocessing.sequence.pad_sequences(array_val, maxlen=max_len) 
        
        y_train = array(df_frac[['Glioma', 'Meningioma', 'Metastasis']][(df_frac['cohort'] == 'train')])
        y_val = array(df_train_val[['Glioma', 'Meningioma', 'Metastasis']][(df_train_val['cohort'] == 'validation')])
        
        #model = get_conv1d_model(n_layers_conv, double_layer, n_filters_conv, filter_size, l1, l2, dropout)
        model = get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer)
        
        el_cb = EarlyStopping(patience=20)
        
        try:
            history = model.fit(X_train, y_train,
                epochs=5000,
                batch_size=32,
                validation_split=0.5,
                callbacks=[el_cb],
                verbose = 0)
            
            auc = roc_auc_score(y_val, model.predict_proba(X_val))
            n_epochs = len(history.epoch)
        except ValueError:
                pass
        
        print('AUC:', str(round(auc,3)))
        bootstrap_aucs.append(auc)

        print('n epochs:', str(round(n_epochs,3)))
        bootstrap_epoch.append(n_epochs)
        
        bootstrap += 1

    time_5_bootstraps = time.time() - start_time
    
    n_bootstraps = bootstrap - 1
    
    print('### Number of bootstraps is', n_bootstraps)
    
    bootstrap_auc_mean = np.mean(bootstrap_aucs)
    bootstrap_auc_std = np.std(bootstrap_aucs)
    
    print('### Mean AUC for grid search', gridsearch, ':', round(bootstrap_auc_mean,2),'±', round(bootstrap_auc_std,3))
    
    bootstrap_epoch_mean = np.mean(bootstrap_epoch)
    bootstrap_epoch_std = np.std(bootstrap_epoch)
    
    print('### Mean n epochs for grid search', gridsearch, ':', round(bootstrap_epoch_mean,2),'±', round(bootstrap_epoch_std,3))
    
    print('')
    
    bootstrap_time_column.append(time_5_bootstraps)
    double_layer_column.append(double_layer)
    dropout_column.append(dropout)
    embedding_dim_column.append(embedding_dim)
    filter_size_column.append(filter_size)
    l1_column.append(l1)
    l2_column.append(l2)
    learning_rate_column.append(learning_rate)
    max_len_column.append(max_len)
    max_pooling_column.append(max_pooling)
    max_words_column.append(max_words)
    n_filters_dense_column.append(n_filters_dense)
    n_filters_conv_column.append(n_filters_conv)
    n_layers_conv_column.append(n_layers_conv)
    n_layers_dense_column.append(n_layers_dense)
    n_params_column.append(model.count_params())
    optimizer_type_column.append(optimizer_type)
    sample_size_column.append(sample_size)
    auc_mean.append(bootstrap_auc_mean)
    auc_std.append(bootstrap_auc_std)
    epochmeans.append(bootstrap_epoch_mean)
    epochstds.append(bootstrap_epoch_std)
    
    gridsearch += 1

    
df_performance = pd.DataFrame(list(zip(sample_size_column, embedding_dim_column, max_len_column, max_words_column, 
                                       n_layers_conv_column, double_layer_column, n_filters_conv_column, filter_size_column,
                                       max_pooling_column, n_layers_dense_column, n_filters_dense_column,
                                       l1_column, l2_column, dropout_column, n_params_column,
                                       auc_mean, auc_std, epochmeans, epochstds, optimizer_type_column, learning_rate_column, bootstrap_time_column)))


df_performance.columns = ['sample_size', 'embedding_dim', 'max_len', 'max_words',
                         'n_layers_conv', 'double_layer', 'n_filters_conv', 'filter_size',
                         'max_pooling', 'n_layers_dense', 'n_filters_dense',
                         'l1_column', 'l2_column', 'dropout', 'n_params',
                         'auc_mean', 'auc_std', 'epochmeans', 'epochstds',
                         'optimizer_type_column', 'learning_rate_column', 'bootstrap_time_column']

target_dir = ### ENTER TARGET DIRECTORY ###

df_performance.to_excel(os.path.join(target_dir, 'hyperparam_tuning_cnn.xlsx'))

df_performance

### 4D. Create optimal parametergrid

In [ ]:
### Because the bootstrapped predictions on the final test required significant computational time,
### the results were computed in 4 batches based on the sample size and merged together.

from sklearn.model_selection import ParameterGrid

param_grid_dict = { 
    'double_layer': [0], #<- final
    'dropout': [0.5], #<- final
    'embedding_dim': [128],# <- final
    'filter_size': [3], #<- final 
    'l1': [0],#<- final
    'l2': [0], #<- final
    'learning_rate': [0.01], #<- final
    'max_len': [1000],#<- final
    'max_pooling' : [1],#<- final
    'max_words': [1000],#<- final
    'n_filters_conv:': [0],#<- final
    'n_filters_dense':[0],#<- final
    'n_layers_conv': [0],#<- final
    'n_layers_dense':[0],#<- final
    'optimizer_type': ['adam'],
    'sample_size': [25, 50, 100, 150, 200, 250, 300, 400, 500, 750, 1000, 1500, 2000, 3000],
                  }

param_grid_list = list(ParameterGrid(param_grid_dict))

parameter_grid = []
for dict in param_grid_list:
    hyperparameters = list(dict.values())
    parameter_grid.append(hyperparameters)

print('Length parameter grid is', len(parameter_grid), 'hyperparameter settings')

parameter_grid

### 4E. Create final performance metrics

In [ ]:
# Both the CNN and non-CNN model architecture can be evaluated by specifying the get_conv1d_model2 function

from keras.layers import Embedding, Flatten, Dense, Input, LSTM, Conv1D, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split, KFold
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import roc_auc_score
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import preprocessing
from keras import regularizers
from keras import optimizers
from keras import layers
from numpy import array
import pandas as pd
import numpy as np
import math
import time

bootstrap_time_column = []
n_layers_dense_column = []
double_layer_column = []
dropout_column = []
embedding_dim_column = []
filter_size_column = []
l1_column = []
l2_column = []
learning_rate_column = []
max_len_column = []
max_pooling_column = []
max_words_column = []
n_filters_dense_column = []
n_filters_conv_column = []
n_layers_conv_column = []
n_params_column = []
optimizer_type_column = []
sample_size_column = []
auc_mean = []
auc_std = []
epochmeans = []
epochstds = []

gridsearch = 1

def get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(dropout))
    if n_layers_conv == 1:
        model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
    if max_pooling == 1:
        model.add(layers.MaxPooling1D(filter_size))
    model.add(layers.GlobalMaxPooling1D())
    if n_layers_dense == 1:
        model.add(Dense(n_filters_dense, activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer=final_optimizer, loss='categorical_crossentropy', metrics=['acc'])
    return model


def square(list):
    return [i ** 2 for i in list]

for double_layer, dropout, embedding_dim, filter_size, l1, l2, learning_rate, max_len, max_pooling, max_words, n_filters_conv, n_filters_dense, n_layers_conv, n_layers_dense, optimizer_type, sample_size in parameter_grid:
    
    print('##### Grid Search', gridsearch,'/', len(parameter_grid))
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(df['stemmed_report'])
    sequences = tokenizer.texts_to_sequences(df['stemmed_report'])
    
    df['vectorized_report'] = sequences 
    
    bootstrap_aucs = []
    bootstrap_auc_stds = []

    
    bootstrap = 1

    if optimizer_type == 'adam':
        final_optimizer = optimizers.Adam(lr=learning_rate)
    elif optimizer_type == 'nadam':
        final_optimizer = optimizers.Nadam(lr=learning_rate)
    
    model = get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer)
    model.summary()

    start_time = time.time()
          
    for n in range(100):
        
        print('--- Bootstrap', bootstrap)

        df_frac = df[(df['cohort'] == 'train')].sample(n = 2*sample_size, replace = True)

        array_train = array(list(df_frac['vectorized_report'][(df_frac['cohort'] == 'train')]))
        array_test = array(list(df['vectorized_report'][(df['cohort'] == 'test')]))

        X_train = preprocessing.sequence.pad_sequences(array_train, maxlen=max_len)
        X_test = preprocessing.sequence.pad_sequences(array_test, maxlen=max_len)
        
        y_train = array(df_frac[['Glioma', 'Meningioma', 'Metastasis']][(df_frac['cohort'] == 'train')])
        y_test = array(df[['Glioma', 'Meningioma', 'Metastasis']][(df['cohort'] == 'test')])
        
        model = get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer)
        
        try:
            history = model.fit(X_train, y_train,
                epochs=50,
                batch_size=32,
                verbose = 0)
            
            auc = roc_auc_score(y_test, model.predict_proba(X_test))
            n_epochs = len(history.epoch)
        except ValueError:
                pass
        
        print('AUC:', str(round(auc,3)))
        bootstrap_aucs.append(auc)

        print('n epochs:', str(round(n_epochs,3)))
        bootstrap_epoch.append(n_epochs)
        
        bootstrap += 1

    time_100_bootstraps = time.time() - start_time
    
    n_bootstraps = bootstrap - 1
    print('### Number of bootstraps is', n_bootstraps)
    
    bootstrap_auc_mean = np.mean(bootstrap_aucs)
    bootstrap_auc_std = np.std(bootstrap_aucs)
    print('### Mean AUC for grid search', gridsearch, ':', round(bootstrap_auc_mean,2),'±', round(bootstrap_auc_std,3))

    bootstrap_epoch_mean = np.mean(bootstrap_epoch)
    bootstrap_epoch_std = np.std(bootstrap_epoch)
    print('### Mean n epochs for grid search', gridsearch, ':', round(bootstrap_epoch_mean,2),'±', round(bootstrap_epoch_std,3))
    
    print('')
    
    bootstrap_time_column.append(time_100_bootstraps)
    double_layer_column.append(double_layer)
    dropout_column.append(dropout)
    embedding_dim_column.append(embedding_dim)
    filter_size_column.append(filter_size)
    l1_column.append(l1)
    l2_column.append(l2)
    learning_rate_column.append(learning_rate)
    max_len_column.append(max_len)
    max_pooling_column.append(max_pooling)
    max_words_column.append(max_words)
    n_filters_dense_column.append(n_filters_dense)
    n_filters_conv_column.append(n_filters_conv)
    n_layers_conv_column.append(n_layers_conv)
    n_layers_dense_column.append(n_layers_dense)
    n_params_column.append(model.count_params())
    optimizer_type_column.append(optimizer_type)
    sample_size_column.append(sample_size)
    auc_mean.append(bootstrap_auc_mean)
    auc_std.append(bootstrap_auc_std)
    epochmeans.append(bootstrap_epoch_mean)
    epochstds.append(bootstrap_epoch_std)
    
    gridsearch += 1

    
df_performance = pd.DataFrame(list(zip(sample_size_column, embedding_dim_column, max_len_column, max_words_column, 
                                       n_layers_conv_column, double_layer_column, n_filters_conv_column, filter_size_column,
                                       max_pooling_column, n_layers_dense_column, n_filters_dense_column,
                                       l1_column, l2_column, dropout_column, n_params_column,
                                       auc_mean, auc_std,
                                       epochmeans, epochstds, optimizer_type_column, learning_rate_column, bootstrap_time_column)))


df_performance.columns = ['sample_size', 'embedding_dim', 'max_len', 'max_words',
                         'n_layers_conv', 'double_layer', 'n_filters_conv', 'filter_size',
                         'max_pooling', 'n_layers_dense', 'n_filters_dense',
                         'l1_column', 'l2_column', 'dropout', 'n_params',
                         'auc_mean', 'auc_std', 'epochmeans', 'epochstds',
                         'optimizer_type_column', 'learning_rate_column', 'bootstrap_time_column']

target_dir = ### ENTER TARGET DIRECTORY ###

df_performance.to_excel(os.path.join(target_dir, 'hyperparam_tuning_cnn.xlsx'))

df_performance

### 4F. Merge final results of the classical CNN model 

In [30]:
### Because the bootstrapped predictions on the final test required significant computational time,
### the results were computed in 4 batches based on the sample size and merged together.


import pandas as pd
import os

path_final_results = ### ENTER SOURCE DIRECTORY ###
df_results_cnn = pd.read_excel(os.path.join(path_final_results, 'part_1.xlsx'))
df_results_part_1 = pd.read_excel(os.path.join(path_final_results, 'part_1.xlsx'))
df_results_part_2 = pd.read_excel(os.path.join(path_final_results, 'part_2.xlsx'))
df_results_part_3 = pd.read_excel(os.path.join(path_final_results, 'part_3.xlsx'))
df_results_part_4 = pd.read_excel(os.path.join(path_final_results, 'part_4.xlsx'))

df_results_cnn = df_results_part_1.append(df_results_part_2, ignore_index=True)
df_results_cnn = df_results_cnn.append(df_results_part_3, ignore_index=True)
df_results_cnn = df_results_cnn.append(df_results_part_4, ignore_index=True)

df_results_cnn = df_results_cnn[['sample_size', 'auc_mean', 'auc_std']]

df_results_cnn.to_excel(os.path.join(path_final_results, 'final_results.xlsx'))
df_results_cnn

,sample_size,auc_mean,auc_std
0,25,0.740113,0.076423
1,50,0.848347,0.061993
2,75,0.881338,0.048086
3,100,0.909558,0.036291
4,150,0.932190,0.028194
5,200,0.936257,0.024811
6,250,0.943332,0.021247
7,300,0.949005,0.019768
8,400,0.956537,0.017690
9,500,0.957508,0.017359


### 4G. Merge final results of the ClinicalTextMiner 

In [29]:
### Because the bootstrapped predictions on the final test required significant computational time,
### the results were computed in 4 batches based on the sample size and merged together.

import pandas as pd
import os

path_final_results = ### ENTER SOURCE DIRECTORY ###
df_results_embed = pd.read_excel(os.path.join(path_final_results, 'part_1.xlsx'))
df_results_part_1 = pd.read_excel(os.path.join(path_final_results, 'part_1.xlsx'))
df_results_part_2 = pd.read_excel(os.path.join(path_final_results, 'part_2.xlsx'))
df_results_part_3 = pd.read_excel(os.path.join(path_final_results, 'part_3.xlsx'))
df_results_part_4 = pd.read_excel(os.path.join(path_final_results, 'part_4.xlsx'))

df_results_embed = df_results_part_1.append(df_results_part_2, ignore_index=True)
df_results_embed = df_results_embed.append(df_results_part_3, ignore_index=True)
df_results_embed = df_results_embed.append(df_results_part_4, ignore_index=True)

df_results_embed = df_results_embed[['sample_size', 'auc_mean', 'auc_std']]

df_results_embed.to_excel(os.path.join(path_final_results, 'final_results.xlsx'))
df_results_embed


,sample_size,auc_mean,auc_std
0,25,0.806184,0.049835
1,50,0.904739,0.030762
2,75,0.944951,0.020864
3,100,0.961468,0.013930
4,150,0.977225,0.007986
5,200,0.982019,0.004516
6,250,0.984145,0.002987
7,300,0.986416,0.002341
8,400,0.987951,0.001850
9,500,0.988919,0.001343
